# Hierarchical Clustering

**Hierarchical clustering** refers to a class of clustering methods that seek to build a **hierarchy** of clusters, in which some clusters contain others. In this assignment, we will explore a top-down approach, recursively bipartitioning the data using k-means.

**Note to Amazon EC2 users**: To conserve memory, make sure to stop all the other notebooks before running this notebook.

## Import packages

In [7]:
# from __future__ import print_function # to conform python 2.x print to python 3.x
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import os
import time
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
%matplotlib inline

## Load the Wikipedia dataset

In [8]:
data = pd.read_csv("/content/drive/MyDrive/FUNIX Progress/MLP304x_0.1-A_EN/data/people_wiki.csv", header=0)

As we did in previous assignments, let's extract the TF-IDF features:

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=200000, stop_words='english', sublinear_tf=True) # 
data_matrix = tf_idf = vectorizer.fit_transform(data["text"])
map_word_to_index = vectorizer.vocabulary_

## Bipartition the Wikipedia dataset using k-means

Recall our workflow for clustering text data with k-means:

1. Load the dataframe containing a dataset, such as the Wikipedia text dataset.
2. Extract the data matrix from the dataframe.
3. Run k-means on the data matrix with some value of k.
4. Visualize the clustering results using the centroids, cluster assignments, and the original dataframe. We keep the original dataframe around because the data matrix does not keep auxiliary information (in the case of the text dataset, the title of each article).

Let us modify the workflow to perform bipartitioning:

1. Load the dataframe containing a dataset, such as the Wikipedia text dataset.
2. Extract the data matrix from the dataframe.
3. Run k-means on the data matrix with k=2.
4. Divide the data matrix into two parts using the cluster assignments.
5. Divide the dataframe into two parts, again using the cluster assignments. This step is necessary to allow for visualization.
6. Visualize the bipartition of data.

We'd like to be able to repeat Steps 3-6 multiple times to produce a **hierarchy** of clusters such as the following:
```
                      (root)
                         |
            +------------+-------------+
            |                          |
         Cluster                    Cluster
     +------+-----+             +------+-----+
     |            |             |            |
   Cluster     Cluster       Cluster      Cluster
```
Each **parent cluster** is bipartitioned to produce two **child clusters**. At the very top is the **root cluster**, which consists of the entire dataset.

Now we write a wrapper function to bipartition a given cluster using k-means. There are three variables that together comprise the cluster:

* `dataframe`: a subset of the original dataframe that correspond to member rows of the cluster
* `matrix`: same set of rows, stored in sparse matrix format
* `centroid`: the centroid of the cluster (not applicable for the root cluster)

Rather than passing around the three variables separately, we package them into a Python dictionary. The wrapper function takes a single dictionary (representing a parent cluster) and returns two dictionaries (representing the child clusters).

In [12]:
def bipartition(cluster, maxiter=400, num_runs=4, seed=None):
    '''cluster: should be a dictionary containing the following keys
                * dataframe: dataframe that concern the current cluster
                * matrix:    the TF-IDF representation of points within the cluster
                * centroid:  centroid for this particular cluster'''
    
    data_matrix = cluster['matrix']
    dataframe   = cluster['dataframe']
    
    # Run k-means on the data matrix with k=2.
    kmeans_model = KMeans(n_clusters=2, max_iter=maxiter, n_init=num_runs, random_state=seed, n_jobs=1)
    kmeans_model.fit_predict(data_matrix)
    centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_
    
    # Divide the data matrix into two parts using the cluster assignments.
    data_matrix_left_child, data_matrix_right_child = data_matrix[cluster_assignment==0], data_matrix[cluster_assignment==1]
    
    # Divide the dataframe into two parts, again using the cluster assignments.
    dataframe_left_child, dataframe_right_child     = dataframe[cluster_assignment==0], dataframe[cluster_assignment==1]
        
    
    # Package relevant variables for the child clusters
    cluster_left_child  = {'matrix': data_matrix_left_child,
                           'dataframe': dataframe_left_child,
                           'centroid': centroids[0]}
    cluster_right_child = {'matrix': data_matrix_right_child,
                           'dataframe': dataframe_right_child,
                           'centroid': centroids[1]}
    
    return (cluster_left_child, cluster_right_child)

The following cell performs bipartitioning of the Wikipedia dataset. Allow 2+ minutes to finish.

Note. For the purpose of the assignment, we set an explicit seed (`seed=0`) to produce identical outputs for every run. In pratical applications, you might want to use different random seeds for all runs.

In [13]:
# %%time
wiki_data = {'matrix': tf_idf, 'dataframe': data} # no 'centroid' for the root cluster
left_child, right_child = bipartition(wiki_data, maxiter=100, num_runs=1, seed=0)

Let's examine the contents of one of the two clusters, which we call the `left_child`, referring to the tree visualization above.

In [14]:
left_child["dataframe"].head()

,URI,name,text
1,<http://dbpedia.org/resource/Alfred_J._Lewy>,Alfred J. Lewy,alfred j lewy aka sandy lewy graduated from un...
2,<http://dbpedia.org/resource/Harpdog_Brown>,Harpdog Brown,harpdog brown is a singer and harmonica player...
3,<http://dbpedia.org/resource/Franz_Rottensteiner>,Franz Rottensteiner,franz rottensteiner born in waidmannsfeld lowe...
4,<http://dbpedia.org/resource/G-Enka>,G-Enka,henry krvits born 30 december 1974 in tallinn ...
5,<http://dbpedia.org/resource/Sam_Henderson>,Sam Henderson,sam henderson born october 18 1969 is an ameri...


And here is the content of the other cluster we named `right_child`.

In [15]:
right_child["dataframe"].head()

,URI,name,text
0,<http://dbpedia.org/resource/Digby_Morrell>,Digby Morrell,digby morrell born 10 october 1979 is a former...
17,<http://dbpedia.org/resource/Paddy_Dunne_(Gael...,Paddy Dunne (Gaelic footballer),paddy dunne was a gaelic football player from ...
21,<http://dbpedia.org/resource/Ceiron_Thomas>,Ceiron Thomas,ceiron thomas born 23 october 1983 is a welsh ...
22,<http://dbpedia.org/resource/Adel_Sellimi>,Adel Sellimi,adel sellimi arabic was born on 16 november 19...
25,<http://dbpedia.org/resource/Vic_Stasiuk>,Vic Stasiuk,victor john stasiuk born may 23 1929 is a reti...


## Visualize the bipartition

We provide you with a modified version of the visualization function from the k-means assignment. For each cluster, we print the top 5 words with highest TF-IDF weights in the centroid and display excerpts for the 8 nearest neighbors of the centroid.

In [20]:
def display_single_tf_idf_cluster(cluster, map_index_to_word):
    '''map_index_to_word: the reverse vocabulary from Vectorizer specifying the mapping betweeen words and column indices'''
    
    wiki_subset   = cluster['dataframe']
    tf_idf_subset = cluster['matrix']
    centroid      = cluster['centroid']
    
    # Print top 5 words with largest TF-IDF weights in the cluster. 
    # Use .argsort and reverse indices to select the highest 5 features index, and refer to the word with map_index_to_word
    top_5_words = centroid.argsort()[::-1]
    print(",".join(str(top_5_words)) + "\n")
    
    # Compute distances from the centroid to all data points in the cluster.
    distances = pairwise_distances(tf_idf_subset, [centroid], metric='euclidean').flatten()
    # compute nearest neighbors of the centroid within the cluster.
    nearest_neighbors = distances.argsort()
    # For 8 nearest neighbors, print the title as well as first 180 characters of text.
    # Wrap the text at 80-character mark.
    for i in range(8):
        text = ' '.join(wiki_subset.iloc[nearest_neighbors[i]]['text'].split(None, 25)[0:25])
        print('* {0:50s} {1:.5f}\n  {2:s}\n  {3:s}'.format(wiki_subset.iloc[nearest_neighbors[i]]['name'],
              distances[nearest_neighbors[i]], text[:90], text[90:180] if len(text) > 90 else ''))
    print('')

Let's visualize the two child clusters:

In [21]:
#map_word_to_index = vectorizer.vocabulary_
map_idx_to_word = {i:w for w, i in map_word_to_index.items()}
display_single_tf_idf_cluster(left_child, map_idx_to_word)

[,1,8,6,3,3,1, ,1,2,4,3,0,9, , ,2,9,4,7,1, ,.,.,., ,1,4,1,8,6,3, , ,3,7,8,1,0, , ,7,6,6,9,8,]

* Yuri Cunza                                         0.96937
  yuri cunza is a hispanicamerican social entrepreneur media professional journalist visual 
  artist business leader humanitarian and community advocate cunza currently serves as presi
* Loren Schoenberg                                   0.97004
  loren schoenberg born july 23 1958 in fair lawn new jersey is a jazz historian writer of l
  iner notes and tenor saxophonist he began playing
* Lawrence W. Green                                  0.97059
  lawrence w green is best known by health education researchers as the originator of the pr
  ecede model and codeveloper of the precedeproceed model which has
* Tomas J. Philipson                                 0.97097
  tomas j philipson is a professor of health economics at the university of chicago with pos
  ts in the harris school of public policy studies department
* Luciano L'Ab

In [22]:
display_single_tf_idf_cluster(right_child, map_idx_to_word)

[,1,6,0,7,9,4, ,1,0,2,2,8,3, ,1,7,6,9,9,1, ,.,.,., ,1,1,9,9,6,0, ,1,1,9,9,5,9, , ,9,9,9,9,9,]

* Todd Williams                                      0.94511
  todd michael williams born february 13 1971 in syracuse new york is a former major league 
  baseball relief pitcher he attended east syracuseminoa high school
* Daniel Healy                                       0.94587
  daniel healy born 3 may 1974 is a former australian rules footballer who played for st kil
  da in the australian football league and for central
* Kevin Nicholson (baseball)                         0.94946
  kevin ronald nicholson born march 29 1976 is a canadian baseball shortstop he played part 
  of the 2000 season for the san diego padres of
* David Beckham                                      0.94968
  david robert joseph beckham obe bkm born 2 may 1975 is an english former professional foot
  baller he has played for manchester united preston north end
* Gord Sherven                                       

The right cluster consists of athletes, or more specifically, team sport players, whereas the left cluster contains the rest. So far, we have a single-level hierarchy consisting of two clusters, as follows:

```
                                           Wikipedia
                                               +
                                               |
                    +--------------------------+--------------------+
                    |                                               |
                    +                                               +
                  Non-athletes                                     Athletes
```

Is this hierarchy good enough? **When building a hierarchy of clusters, we must keep our particular application in mind.** For instance, we might want to build a **directory** for Wikipedia articles. A good directory would let you quickly narrow down your search to a small set of related articles. The categories of athletes and non-athletes are too general to facilitate efficient search. For this reason, we decide to build another level into our hierarchy of clusters with the goal of getting more specific cluster structure at the lower level. To that end, we subdivide both the `athletes` and `non-athletes` clusters.

## Perform recursive bipartitioning

### Cluster of athletes and artists

To help identify the clusters we've built so far, let's give them easy-to-read aliases:

In [23]:
non_athletes   = left_child
athletes       = right_child

Using the bipartition function, we produce two child clusters of the athlete cluster:

In [24]:
# Bipartition the cluster of athletes and artists
left_child_athletes, right_child_athletes = bipartition(athletes, maxiter=100, num_runs=6, seed=1)

The left child cluster mainly consists of _pure_ athletes, e.g track & fields or olympic participants:

In [25]:
display_single_tf_idf_cluster(left_child_athletes, map_idx_to_word)

[, ,2,3,4,2,3, ,1,0,2,2,8,3, ,1,0,8,8,8,1, ,.,.,., ,1,3,0,0,5,1, ,1,3,0,0,5,0, , ,9,9,9,9,9,]

* Dave Ford                                          0.89552
  david alan ford born december 29 1956 is a former major league baseball pitcher for the ba
  ltimore orioles born in cleveland ohio ford attended lincolnwest
* Justin Knoedler                                    0.89620
  justin joseph knoedler born july 17 1980 in springfield illinois is a former major league 
  baseball catcherknoedler was originally drafted by the st louis cardinals
* Steve Springer                                     0.89685
  steven michael springer born february 11 1961 is an american former professional baseball 
  player who appeared in major league baseball as a third baseman and
* James Baldwin (baseball)                           0.89748
  james j baldwin jr born july 15 1971 is a former major league baseball pitcher he batted a
  nd threw righthanded in his 11season career he
* Dom Zanni                

On the other hand, the right child cluster consists mainly of sport players (baseball, footballer, etc.):

In [26]:
display_single_tf_idf_cluster(right_child_athletes, map_idx_to_word)

[,1,7,6,9,9,1, ,1,6,0,7,9,4, ,1,4,0,1,8,8, ,.,.,., ,1,2,0,7,8,5, ,1,2,0,7,8,2, , ,9,9,9,9,9,]

* Daniel Healy                                       0.94491
  daniel healy born 3 may 1974 is a former australian rules footballer who played for st kil
  da in the australian football league and for central
* David Beckham                                      0.94828
  david robert joseph beckham obe bkm born 2 may 1975 is an english former professional foot
  baller he has played for manchester united preston north end
* Frank Lampard                                      0.94914
  frank james lampard born 20 june 1978 is an english professional footballer who plays as a
   central midfielder attacking midfielder or defensive midfielder for manchester
* Gord Sherven                                       0.94980
  gordon r sherven born august 21 1963 in gravelbourg saskatchewan and raised in mankota sas
  katchewan is a retired canadian professional ice hockey forward who played
* Jason Robe

Our hierarchy of clusters now looks like this:
```
                                           Wikipedia
                                               +
                                               |
                    +--------------------------+--------------------+
                    |                                               |
                    +                                               +
                Non-athletes                                    Athletes
                                                                    +
                                                                    |
                                                         +----------+----------+
                                                         |                     |
                                                         |                     |
                                                         +                     |
                                                     Athletic-typed          Team-typed
```

Should we keep subdividing the clusters? If so, which cluster should we subdivide? To answer this question, we again think about our application. Since we organize our directory by topics, it would be nice to have topics that are about as coarse as each other. For instance, if one cluster is about baseball, we expect some other clusters about football, basketball, volleyball, and so forth. That is, **we would like to achieve similar level of granularity for all clusters.**

Both the athletic sport and team sport node can be subdivided more, as both have non-overlapping subdomains of specific sports (track/marathon/cross-country runs, or baseball/football/basketball, etc.). Let's explore subdividing the athletes cluster further to produce finer child clusters.

Let's give the clusters aliases as well:

In [27]:
athletic    = left_child_athletes
team_sport  = right_child_athletes

### Cluster of athletic

In answering the following quiz question, take a look at the topics represented in the top documents (those closest to the centroid), as well as the list of words with highest TF-IDF weights.

Let us bipartition the cluster of athletes.

In [28]:
left_child_athletic, right_child_athletic = bipartition(athletic, maxiter=100, num_runs=6, seed=1)

In [29]:
display_single_tf_idf_cluster(left_child_athletic, map_idx_to_word)
display_single_tf_idf_cluster(right_child_athletic, map_idx_to_word)

[, ,2,3,4,2,3, ,1,3,9,7,0,5, ,1,0,2,2,8,3, ,.,.,., ,1,3,1,3,1,0, ,1,3,1,3,0,9, , ,9,9,9,9,9,]

* Dave Ford                                          0.86081
  david alan ford born december 29 1956 is a former major league baseball pitcher for the ba
  ltimore orioles born in cleveland ohio ford attended lincolnwest
* Dom Zanni                                          0.87621
  dominick thomas zanni born march 1 1932 in the bronx new york is a former professional bas
  eball player a righthanded pitcher he pitched in 111
* Jamie Brewington                                   0.87982
  jamie chancellor brewington born september 28 1971 in greenville north carolina is an amer
  ican former major league baseball player the righthanded pitcher played for the
* Rick Williams (baseball b. 1952)                   0.88671
  richard allen williams born november 9 1952 at merced california is a retired american pro
  fessional baseball player he was a 6 ft 1 in 185
* Fred Norman                     

**Quiz Question**. Which diagram best describes the hierarchy right after splitting the `athletic` cluster? Refer to the quiz form for the diagrams.

**Caution**. The granularity criteria is an imperfect heuristic and must be taken with a grain of salt. It takes a lot of manual intervention to obtain a good hierarchy of clusters.

* **If a cluster is highly mixed, the top articles and words may not convey the full picture of the cluster.** Thus, we may be misled if we judge the purity of clusters solely by their top documents and words. 
* **Many interesting topics are hidden somewhere inside the clusters but do not appear in the visualization.** We may need to subdivide further to discover new topics. In this very instance, subdividing the `athletic` cluster led to the discovery of the `golfers` child cluster.

### Cluster of non-athletes

Now let us subdivide the cluster of non-athletes.

In [30]:
#%%time 
# Bipartition the cluster of non-athletes
left_child_non_athletes, right_child_non_athletes = bipartition(non_athletes, maxiter=100, num_runs=3, seed=1)

In [31]:
display_single_tf_idf_cluster(left_child_non_athletes, map_idx_to_word)

[,1,2,1,2,3,6, , ,6,5,8,8,4, ,1,2,4,3,0,9, ,.,.,., ,1,0,2,7,5,8, ,1,7,5,1,5,9, ,1,3,1,0,0,5,]

* Loren Schoenberg                                   0.96441
  loren schoenberg born july 23 1958 in fair lawn new jersey is a jazz historian writer of l
  iner notes and tenor saxophonist he began playing
* Craig Pruess                                       0.96732
  craig pruess born 1950 is an american composer musician arranger and gold platinum record 
  producer who has been living in britain since 1973 his career
* Wilson McLean                                      0.96874
  wilson mclean born 1937 is a scottish illustrator and artist he has illustrated primarily 
  in the field of advertising but has also provided cover art
* Bob Mintzer                                        0.96948
  bob mintzer is a jazz saxophonist composer arranger and big band leader based in los angel
  es californiabob mintzer born january 27 1953 and a native
* Peter Combe                                     

In [32]:
display_single_tf_idf_cluster(right_child_non_athletes, map_idx_to_word)

[,1,8,6,3,3,1, ,1,6,2,4,3,0, ,1,1,3,9,3,8, ,.,.,., ,1,3,5,2,6,2, , ,6,3,9,6,1, ,1,9,9,9,9,9,]

* Lawrence W. Green                                  0.95987
  lawrence w green is best known by health education researchers as the originator of the pr
  ecede model and codeveloper of the precedeproceed model which has
* John P. White                                      0.96140
  john patrick white born february 27 1937 is an american university professor and a former 
  government official who served in the clinton administration he was
* Barry Sullivan (lawyer)                            0.96161
  barry sullivan is a chicago lawyer and as of july 1 2009 the cooney conway chair in advoca
  cy at loyola university chicago school of law
* David Anderson (British Columbia politician)       0.96203
  david a anderson pc oc born august 16 1937 in victoria british columbia is a former canadi
  an cabinet minister educated at victoria college in victoria
* Andrew Fois                           

This clusters are not as clear-cut as the previous one, but the left cluster has a tendency to show artists and musicians, and the right one to show politicians and government officials.

Let's divide them further.

In [33]:
artists = left_child_non_athletes
politicians_etc = right_child_non_athletes

**Quiz Question**. Let us bipartition the clusters `artists` and `politicians`. Which diagram best describes the resulting hierarchy of clusters for the non-athletes? Refer to the quiz for the diagrams.

**Note**. Use `maxiter=100, num_runs=6, seed=1` for consistency of output.

In [35]:
left_child_non_artists, right_child_non_artists = bipartition(artists, maxiter=100, num_runs=6, seed=1)

In [37]:
left_child_non_politicians, right_child_non_politicians = bipartition(politicians_etc, maxiter=100, num_runs=6, seed=1)

In [38]:
display_single_tf_idf_cluster(left_child_non_artists, map_idx_to_word)
display_single_tf_idf_cluster(right_child_non_artists, map_idx_to_word)

[,1,2,1,2,3,6, , ,1,3,0,5,0, , ,2,2,4,7,2, ,.,.,., , ,8,1,9,3,4, , ,8,1,9,3,5, , ,9,9,9,9,9,]

* Dan Siegel (musician)                              0.95216
  dan siegel born in seattle washington is a pianist composer and record producer his earlie
  r music has been described as new age while his more
* Graham Ord                                         0.95625
  graham ord born 22 march 1961 is an english musician and songwriter he has garnered respec
  t internationally as a fine musician and engaging communicator his
* Bruno Mars                                         0.95657
  peter gene hernandez born october 8 1985 professionally known by his stage name bruno mars
   is an american singer songwriter record producer voice actor and
* Frank Mills                                        0.95731
  frank mills born june 27 1942 in montreal quebec is a canadian pianist and recording artis
  t best known for his solo instrumental hit music box
* Prince (musician)                       

In [39]:
display_single_tf_idf_cluster(left_child_non_politicians, map_idx_to_word)
display_single_tf_idf_cluster(right_child_non_politicians, map_idx_to_word)

[,1,3,5,8,1,1, , ,5,8,6,1,0, , ,5,8,6,0,4, ,.,.,., ,1,2,2,9,3,2, ,1,2,2,9,2,9, , , , , , ,0,]

* Don Bell                                           0.94337
  donald h bell born march 10 1942 in new westminster british columbia is a canadian politic
  ian he is currently serving as a councillor for the
* Doug Lewis                                         0.94351
  douglas grinslade doug lewis pc qc born april 17 1938 is a former canadian politician a ch
  artered accountant and lawyer by training lewis entered the
* Marcelle Mersereau                                 0.94488
  marcelle mersereau born february 14 1942 in pointeverte new brunswick is a canadian politi
  cian a civil servant for most of her career she also served
* Kayee Griffin                                      0.94583
  kayee frances griffin born 6 february 1950 is an australian politician and former australi
  an labor party member of the new south wales legislative council serving
* Marcel Mass%C3%A9                 